<a href="https://colab.research.google.com/github/Sylver640/ADA-Informes/blob/main/Informe_BellmanFord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Descripción del problema (camino más corto)
---
**Entrada**: Un grafo dirigido $G=(V,E)$, un vértice fuente $s\in V$, y un valor real $l_e \geq 0$ asociado a cada arco $e\in E$.

**Salida**: La distancia más corta $dist(s,v)$ para cada vértice $v\in V$.

----

Continuando la linea de problemas clásicos en la teoría de grafos, era imposible dejar fuera al conocido **camino más corto**, en este caso con una fuente única. En términos generales, esta versión del problema tiene como objetivo encontrar la **distancia más corta** entre un vértice inicial o fuente $s$ y todos los nodos del grafo dirigido $G = (V, E)$. Definiremos la distancia o largo de un camino como la suma de sus arcos. 

Por ejemplo, en el siguiente grafo, las distancias más cortas entre $s$ y el resto de los nodos son: $distancia(s,s)=0$, $distancia(s,v)=1$, $distancia(s,w)=3$ y $distancia(s,t)=6$.

![image](https://chartreuse-goal-d5c.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F9353642d-b383-4a2c-96b0-479eb5240879%2FUntitled.png?table=block&id=0491d913-c844-4de0-a5a4-9b1bfe386320&spaceId=4f8bebe4-a843-44d2-b6ee-51e2006a90d1&width=2000&userId=&cache=v2)

Un algoritmo que logra entregar una solución óptima a esta problemática es el conocido **algoritmo de Dijkstra**, el cual es capaz de encontrar la distancia más corta en grafos con solamente arcos positivos. 

No obstante, esta problemática no se limita a tan solo aplicaciones que sean de esta índole. Por lo tanto, una versión revisada del problema que permita esto tendría las siguientes características:

---

**Entrada**: Un grafo dirigido $G=(V,E)$, un vértice fuente $s\in V$, y un valor real $l_e$ asociado a cada arco $e\in E$.

**Salida**: Una de las siguientes opciones:

1. La distancia más corta $dist(s,v)$ para cada vértice $v\in V$.
2. Una declaración indicando que $G$ contiene un ciclo negativo.

---

Así tenemos el mismo problema, el cual nuevamente tiene el objetivo de encontrar el camino más corto desde una fuente única. Esto sobre grafos dirigidos y con arcos **positivos o negativos**. Sin embargo, con algo más de versatilidad tenemos una limitante, y estos son los llamados **ciclos negativos**, los cuales impiden entregar una correcta solución al problema, puesto que podríamos iterar indefinidamente en este ciclo reduciendo infinitamente el largo del camino. En esta versión del problema, un algoritmo capaz de solucionarlo es el **algoritmo de Bellman-Ford**, el cual trabaja bajo el paradigma de la programación dinámica.

#2. Algoritmos
A continuación, se presentarán las implementaciones de los dos algoritmos mencionados anteriormente: Dijkstra y Bellman-Ford. Para poder trabajar con ellos, se utilizará el generador de instancias propuesto en el siguiente bloque de código.

In [4]:
import random

### Generadores de instancia ###

def is_valid_edge(generated_edges: dict, i: int, j: int):
    return i != j and not generated_edges.get((i, j), None) and not generated_edges.get((j, i), None)

def instance_generator_bellman(n: int):
    """
        Input: cantidad de vértices
        Output: una lista que contiene todos los arcos y el número del vértice fuente (la función retorna dos variables).
        Los arcos vienen en la forma (i, j, weight), donde i es el vértice origen del arco y j el vértice al que apunta el arco, mientras que weight es su peso.
    """
    graph = []
    nodes = random.sample(range(0, n), n)
    unvisited_nodes = random.sample(range(0, n), n)
    
    generated_edges = {}
    for i in nodes:
        rand = random.sample(nodes, random.randint(1, 3))

        for j in rand:
            edge = (i, j)
            edge_with_weight = (i, j, random.randint(1, 100))
            
            if generated_edges.get((edge[1], edge[0]), None):
                continue
            
            if i == j:
                new_vertice = None
                iterations = 0
                while new_vertice is None and iterations < 250:
                    iterations += 1
                    number = random.randint(0, n - 1)
                    if is_valid_edge(generated_edges, i, number):
                        new_vertice = number

                if iterations >= 250:
                    return instance_generator_bellman(n)
                
                edge = (i, new_vertice)
                edge_with_weight = (i, new_vertice, random.randint(-25, 100)) # -25 y 100 corresponde a los límites de los pesos, puede cambiarlos.
            
            graph.append(edge_with_weight)
            generated_edges[edge] = edge

            if edge_with_weight[1] in unvisited_nodes:
                unvisited_nodes.remove(edge_with_weight[1])

    for i in unvisited_nodes:
        valid_edge = False
        iterations = 0
        while not valid_edge and iterations < 250:
            iterations += 1
            m = random.randint(0, n - 1)
            if is_valid_edge(generated_edges, m, i):
                valid_edge = True
                edge = (m, i)
                edge_with_weight = (m, i, random.randint(-25, 100)) # -25 y 100 corresponde a los límites de los pesos, puede cambiarlos.
                graph.append(edge_with_weight)
                generated_edges[edge] = edge

        if iterations >= 250:
            return instance_generator_bellman(n)

    return graph, graph[0][0]

def instance_generator_dijkstra(n: int):
    """
        Input: cantidad de vértices
        Output: una lista que contiene todos los arcos y el número del vértice fuente (la función retorna dos variables).
        Los arcos vienen en la forma (i, j, weight), donde i es el vértice origen del arco y j el vértice al que apunta el arco, mientras que weight es su peso.
    """
    graph = []
    nodes = random.sample(range(0, n), n)
    unvisited_nodes = random.sample(range(0, n), n)
    
    generated_edges = {}
    for i in nodes:
        rand = random.sample(nodes, random.randint(1, 3))

        for j in rand:
            edge = (i, j)
            edge_with_weight = (i, j, random.randint(1, 100))
            
            if generated_edges.get((edge[1], edge[0]), None):
                continue
            
            if i == j:
                new_vertice = None
                iterations = 0
                while new_vertice is None and iterations < 250:
                    iterations += 1
                    number = random.randint(0, n - 1)
                    if is_valid_edge(generated_edges, i, number):
                        new_vertice = number

                if iterations >= 250:
                    return instance_generator_dijkstra(n)
                
                edge = (i, new_vertice)
                edge_with_weight = (i, new_vertice, random.randint(0, 100)) # -25 y 100 corresponde a los límites de los pesos, puede cambiarlos.
            
            graph.append(edge_with_weight)
            generated_edges[edge] = edge

            if edge_with_weight[1] in unvisited_nodes:
                unvisited_nodes.remove(edge_with_weight[1])

    for i in unvisited_nodes:
        valid_edge = False
        iterations = 0
        while not valid_edge and iterations < 250:
            iterations += 1
            m = random.randint(0, n - 1)
            if is_valid_edge(generated_edges, m, i):
                valid_edge = True
                edge = (m, i)
                edge_with_weight = (m, i, random.randint(0, 100)) # -25 y 100 corresponde a los límites de los pesos, puede cambiarlos.
                graph.append(edge_with_weight)
                generated_edges[edge] = edge

        if iterations >= 250:
            return instance_generator_dijkstra(n)

    return graph, graph[0][0]

##2.1 Algoritmo de Dijkstra
También llamado "algoritmo de caminos mínimos", éste fue concebido por el científico en computación Edsger Dijkstra en 1956, y publicado en 1959. Soluciona la primera versión del problema, es decir, solo trabaja con arcos positivos.

In [10]:
def dijkstraAlgorithm(G, s, verbose, visualize):
  distancias = [1e7] * len(G)
  distancias[s] = 0
  
  return 0

#Ejemplo
n_dijkstra = random.randint(5,25)
G_dijkstra, fuente_dijkstra = instance_generator_dijkstra(n_dijkstra)

print(f"Dijkstra: {G_dijkstra}")
print(f"Fuente: {fuente_dijkstra}")

dijkstraAlgorithm(G_dijkstra, fuente_dijkstra, verbose = False, visualize = False)

Dijkstra: [(15, 20, 87), (15, 5, 46), (8, 5, 23), (8, 20, 41), (19, 4, 22), (19, 10, 27), (16, 3, 31), (1, 12, 70), (1, 4, 11), (12, 3, 65), (12, 14, 53), (9, 16, 65), (21, 3, 44), (21, 19, 43), (13, 15, 23), (13, 9, 31), (20, 6, 10), (20, 9, 81), (5, 1, 2), (3, 9, 62), (3, 20, 62), (3, 18, 98), (6, 3, 85), (6, 21, 31), (6, 13, 52), (2, 14, 95), (2, 3, 58), (11, 2, 73), (11, 0, 7), (10, 7, 39), (4, 14, 65), (17, 14, 5), (17, 11, 19), (17, 11, 46), (0, 18, 58), (0, 8, 97), (0, 9, 79), (18, 1, 49), (18, 14, 41), (7, 12, 71), (14, 21, 15), (14, 20, 41), (14, 7, 81), (16, 17, 93)]
Fuente: 15
0


0

###2.1.1 Descripción del algoritmo
Éste recibe un grafo dirigido de arcos positivos $G = (V, E)$ y un vértice fuente $s \in V$

###2.1.2 Ejemplo

###2.1.3 Visualización del camino más corto encontrado

###2.1.4 Ejecución del algoritmo paso a paso (`verbose = True`)

##2.2 Algoritmo de Bellman-Ford

In [ ]:
def bellmanFordAlgorithm(G, fuente):
  return 0

#Ejemplo
n_bellman = random.randint(5,25)
G_bellman, fuente_bellman = instance_generator_bellman(n_bellman)

print(f"Bellman: {G_bellman}")
print(f"Fuente: {fuente_bellman}")

Bellman: [(1, 5, 87), (1, 12, 31), (3, 13, 58), (3, 18, 98), (17, 12, 15), (17, 10, 56), (20, 5, 73), (20, 0, 24), (20, 19, 36), (9, 18, 76), (10, 2, 45), (10, 18, 50), (10, 16, 14), (0, 3, 6), (0, 5, 30), (0, 2, 80), (15, 8, 26), (15, 0, 90), (18, 7, 60), (18, 2, 39), (7, 20, -2), (7, 4, 71), (7, 21, 32), (16, 2, 28), (16, 8, 38), (14, 0, 21), (14, 16, 12), (8, 18, 18), (8, 12, 53), (21, 15, 85), (12, 3, 93), (4, 20, 80), (11, 16, 13), (11, 13, -12), (11, 7, 95), (6, 2, 27), (6, 13, 57), (19, 21, 78), (13, 20, 37), (13, 10, 25), (5, 21, 36), (18, 17, 91), (10, 11, -22), (19, 1, 25), (8, 9, 57), (14, 6, 55), (1, 14, 91)]
Fuente: 1


###2.2.1 Descripción del algoritmo

###2.2.2 Ejemplo

###2.2.3 Visualización del camino más corto encontrado

###2.2.4 Ejecución del algoritmo paso a paso (`verbose = True`)

#3. Correctitud del algoritmo de Bellman-Ford

#4. Tiempo de ejecución

#5. Experimentos